# Preparation of Dicke States

In this tutorial, we will show you different routines for the preparation of Dicke States and how to implement them in Qrisp! They are all [JASP](../../reference/Jasp/index.rst) compatible, so you can seamlessly integrate them into your realtime computations as a subroutine. 

The two main approaches we will look at are [an efficient deterministic algorithm by Jeffery Yu et al.](https://arxiv.org/abs/2411.03428) , and [a deterministic algorithm by Bärtschi et al.](https://arxiv.org/abs/1904.07358), as well as the  [more efficient version](https://arxiv.org/abs/2112.12435) of the latter.   

But before we jump into their preparation, let us first answer the questions: **What are Dicke states? And what do we use them for?**

## Definition and utility of Dicke States

The preparation of Dicke states is an important and highly investigated topic in the field of quantum computation due to to their range of applications. 
This ranges from error correction techniques in the ISD algorithm to optimization tasks solved with [DQI](https://arxiv.org/abs/2408.08292) or [QAOA](https://arxiv.org/pdf/2207.10555). 
Furthermore, these states are native to quantum optics systems due to their relation to spin systems, 
and can be found in quantum communication applications. 

Formally, Dicke states are defined as follows:

Dicke states are characterized by there constant Hamming-weight. A Dicke state with $n$ qubits and Hamming-weight $\text{wt}(x)=k$ is defined as

$ \ket{D_{k}^{n}} = \binom{n}{k}^{-\frac{1}{2}} \sum_{x \in \{ 0,1 \}^{\otimes n},  \text{wt} ( x ) = k  } \ket{x} $


An example for this is the Dicke state $\ket{D_{2}^{4}} = \frac{1}{\sqrt{6}} ( \ket{1100} + \ket{1010} + \ket{1001} +\ket{0110} + \ket{0101} + \ket{0011} )$, consisting of 4 qubits with Hamming-weight $wt(x)=2$.  
It can also be defined in direct relation to a spin system, see p.e. the [related wikipedia article](https://en.wikipedia.org/wiki/Dicke_state).


## Preparation algorithms

After the short overview on their definition and applications, let's now dive into the established algorithms for Dicke state preparation! 
We will start with the probablistic which offers a cool physics-inspired solution! 

### Probablistic approach

The first algorithm we want to highlight here is [Efficient preparation of Dicke states (2024)](https://arxiv.org/abs/2411.03428) by Jeffery Yu et al., from the University of Maryland. 
The algorithm utilizes mid-circuit Hamming-weight measurements and feedback to prepare Dicke states by incorporating adaptively-chosen global rotations.

<img src="../../_static/dicke_pseudo_algo.png" class="align-center" width="800" alt="Dicke state pseudo algorithm" />



The goal of the algorithm is to prepare a Dicke state $\ket{j, m_t}$ for a desired target value of $m_t$ , starting from the initial product state $\ket{j, j} = \ket{0}^{\otimes n}$.
One starts by performing a uniform rotation $e^{-i\theta J_Y}$ for some angle $\theta$ and measuring $J_Z$, the eigenvalue in the Pauli Z-basis.
If $m = m_t$ is measured, the procedure ends. Otherwise, we iterate, choosing subsequent rotation angles $\theta$ based on the prior outcome of the measurement of $J_Z$.
The rotation angles $\theta$ are chosen to maximize the overlap of the current state with the target Dicke state on each iteration.
The pseudocode for these iterations is shown in the figure above.

<img src="../../_static/dicke_rotations.png" class="align-center" width="800" alt="Dicke state rotations" />

The angle $\theta_{m_t, m}$ is chosen such that it maximizes the overlap of the Husimi–Q distributions (for more information refer to the paper) of the rotated state and the target Dicke state in the limit of large $j$.
We reach this maximum when the corresponding ring distributions intersect at a point where they share the same tangent vector, as shown in the figure above.
With this condition, we can derive the rotation angle to be:

$ \theta_{m_t, m} = \arcsin \left( \frac{m_r \cdot m_t - m_t \cdot r_m}{r_0^2} \right) $

The paper further explains how the collective Hamming weight measurements may be directly implemented on an ensemble of $n$ atomic qubits, in which one of the two qubit states is coupled to a single-mode cavity.
In conclusion, we arrive at an algorithm for preparing Dicke states with depth and width logarithmic in the number of qubits, using only sequences of global single-qubit rotations and collective native Hamming weight measurements.

### Implementation 

For the implementation, we will resort back to circuit representation of the aforementioned components, namely the Hamming-weight measurements and the rotations around the Bloch sphere. 

As explained in the paper, one of the facilitators for this algorithm is the collective Hamming-weight measurement, so let us start by creating that. 
It is taken from a [paper on Shallow Quantum Circuits](https://arxiv.org/pdf/2404.06052) and gives a straight forward 
scheme. We first create the necessary amount of ancilla qubits, and then apply the routine based on controlled rotations and an inverse [Quantum Fourier transformation](../../reference/Primitives/QFT.rst).

In [14]:
import jax.numpy as jnp 
from qrisp import QuantumFloat, h, cx, rz, jrange, QFT, ry, measure, q_while_loop, terminal_sampling


def collective_hamming_measurement(qf, n):

    # create ancillas and put the in superposition
    n_anc = jnp.ceil(jnp.log2(n)+1).astype(int)
    ancillas = QuantumFloat(n_anc)
    h(ancillas)

    # controlled rz rotations 
    for i in jrange(n_anc):
        rz(2*n*jnp.pi/(2**(n_anc+1-i)),ancillas[i])
        
    for i in jrange(n_anc):
        for k in jrange(n):
            cx(ancillas[n_anc-i-1], qf[k])
        for k in jrange(n):
            rz(-2*jnp.pi/(2**(i+2)), qf[k])
        for k in jrange(n):
            cx(ancillas[n_anc-i-1], qf[k])

    # inverse QFT for correct representation
    QFT(ancillas,inv=True)
    
    return ancillas

With that out of the way, let's get back to the implementation of the algorithm, which emerges naturally from the pseudocode provided.
It is already [jaspified](../../reference/Jasp/index.rst) and intended to be called with the [@terminal_sampling](../../reference/Jasp/Simulation%20Tools/Terminal%20Sampling.rst) decorator (or an adaption with makes use of the ``terminal_sampling`` simulator optimization).

To reiterate: the procedure performs iterative [ry()](../../reference/Primitives/generated/qrisp.ry.rst)-rotations, where the rotation angle is adaptively chosen based on the [collective_hamming_measurement()](../../reference/Primitives/DickeStates.rst) of previous iteration.
We stop once we measure the correct Hamming-weight. 
In *JASP*-terms, this is achieved by wrapping the rotate-and-measure procedure in a [q_while_loop()](../../reference/Jasp/Control%20Flow/Prefix%20Control.rst#qrisp.jasp.q_while_loop). This jaspified version of a quantum while-loop requires a condition function ``cond_fun()`` with a ``bool`` (or [QuantumBool](../../reference/Quantum%20Types/QuantumBool.rst)) return, and a body function ``body_fun()``.
The ``cond_fun()`` checks whether the "while" condition is still true, while the ``body_fun()`` performs the iterative quantum operations.

Let us investigate the ``body_fun()`` first. We directly translate what is proposed proposes into code. First, we perform some arithmetic to find the updated rotation angle.
Then, we apply the corrected [ry()](../../reference/Primitives/generated/qrisp.ry.rst)-rotations. And finally, we perform the [collective_hamming_measurement()](../../reference/Primitives/DickeStates.rst) to gather information about our Hamming-weight overlap. 

In [15]:

# define a placeholder QuantumFloat and placeholder m_t desired Hamming-weight
qf = QuantumFloat(4)
m_t = 2

j = qf.size 

# algebra from paper for initial values
r_mt = jnp.sqrt(j*(j+1)-m_t**2)
r_0 = jnp.sqrt(j*(j+1))

def body_fun(val):
    # assign initial values
    m_t, qf1, theta, j, m = val
    # algebra from paper
    r_m = jnp.sqrt(j * (j+1) - m.astype(float) **2)
    theta = jnp.asin((m * r_mt - m_t.astype(float) * r_m) /r_0**2)

    # rotation towards desired state
    for t in jrange(j):
        ry(theta, qf1[t])

    # collective hamming weight measurement and uncomputation
    ancillas = collective_hamming_measurement(qf1,j)
    m = measure(ancillas)
    # delete ancillas
    ancillas.delete()

    return m_t, qf1, theta, j, m.astype(int)  

The ``cond_fun()`` is very simple. All it does is check whether the result from the Hamming-weight measurement (described by ``val[-1]``) 
is equivalent to the one we are looking for (which is given by ``val[0]``). If yes, we stop the loop.

In [16]:
def cond_fun(val):
    return val[0] != val[-1]

Putting it all together, the main function ``iterative_dicke_state_sampling()`` reduces to seven lines of code, with the [q_while_loop()](../../reference/Jasp/Control%20Flow/Prefix%20Control.rst#qrisp.jasp.q_while_loop) being the central ingredient.

In [17]:
def iterative_dicke_state_sampling(qf, m_t):
    
    j = qf.size 

    # algebra from paper for initial values
    r_mt = jnp.sqrt(j*(j+1)-m_t**2)
    r_0 = jnp.sqrt(j*(j+1))

    # body_fun - insert it here
    
    # cond_fun - insert it here

    thet_0 = 0
    
    m_t, qf1, thet_0, j, m  = q_while_loop(cond_fun, body_fun, (m_t, qf,thet_0 ,j,j))
    
    return qf1



To give a final example, this what the code looks like to create the aforementioned $\ket{D_{2}^{4}}$ state:

In [18]:
    #We instantiate a QuantumVariable with 4 qubits from this create the Dicke state with Hamming weight 2
@terminal_sampling
def main():
        
    n = 4
    k = 2
    qv_iter = QuantumFloat(n)
    qv_iter = iterative_dicke_state_sampling(qv_iter,k)

    return qv_iter

dicke_qv = main()
print(dicke_qv)
# returns (1 / \sqrt{6} ( |1100> + |1010> + |1001> + |0110> + ||0101> + |0011> ) encoded as QuantumFloats

{12.0: 0.1666667014360459, 3.0: 0.16666667163372084, 5.0: 0.16666665673255832, 6.0: 0.16666665673255832, 9.0: 0.16666665673255832, 10.0: 0.16666665673255832}


And thats it! All you need to create a Dicke state with the *JASP* compilation pipeline. 

Let us now continue with the deterministic approach.

## Deterministic approach 

The other algorithm of interest is [Deterministic Preparation of Dicke States (2019)](https://arxiv.org/abs/1904.07358) and its more efficient variation [A Divide-and-Conquer Approach to Dicke State preparation (2021)](https://arxiv.org/abs/2112.12435>). 

The second algorithm mentioned is a divide-and-conquer adaption based on the first one, as the name would suggest. So let us start with the first paper. 

In it the authors make use of *split & cyclic shift* unitaries, which are then applied inductively in a cascade. In the following, we will show you how 
the basic components are implemented and how these unitary calls are structed in terms of Qrisp code.

For an in-depth explanation on how these unitaries emerge and their action on a quantum state, please refer to the original paper. 

The aforementioned unitary is given by the function [split_cycle_shift()](../../reference/Primitives/DickeStates.rst), which receives a [QuantumVariable](../../reference/Core/QuantumVariable.rst) ``qv``. 
Additionally, two integers,  ``highIndex`` and ``lowIndex``, indicate the preparation steps, as seen in original algorithm.

Some caveats: 

This implementation is *JASP* ready. It therefore makes use of the [jrange()](../../reference/Jasp/Control%20Flow/jrange.rst) iterator. In the paper, the iteration is conducted in reverse, i.e. from the lowest to the highest index. 
In a normal ``range()`` iterator, you would just set ``step =-1`` for this behaviour; [jrange()](../../reference/Jasp/Control%20Flow/jrange.rst) does not allow for this. Instead, we embed the whole construct in an [InversionEnvironment](../../reference/Quantum%20Environments/InversionEnvironment.rst)  to reverse the loop.

Additionally, you may notice some logic checks using the ``ctrl_bool`` variables. This replaces ``if``-statement usage in *JASP* mode, so make good use of that when **jaspifying** your Qrisp code! 


In [19]:
from qrisp import control, invert

def split_cycle_shift(qv, highIndex, lowIndex):

        with invert():
            # reversed jrange
            for i in jrange(lowIndex): 

                index = highIndex - i 
                param = 2 * jnp.arccos(jnp.sqrt((highIndex - index + 1 ) /(highIndex)) )

                ctrL_bool = index == highIndex
                ctrL_bool_false = index != highIndex

                # conditional application of the cx and c-ry rotations 
                with control(ctrL_bool):
                    cx(qv[highIndex - 2], qv[highIndex-1]) 
                    with control( qv[highIndex-1] ):
                        ry(param, qv[highIndex - 2])
                    cx(qv[highIndex - 2], qv[highIndex -1])
                
                with control(ctrL_bool_false):
                    cx(qv[index -2], qv[highIndex-1]) 
                    with control([qv[highIndex -1],qv[index -1]]):
                        ry(param, qv[index - 2])
                    cx(qv[index -2], qv[highIndex-1]) 

These *split & cyclic shift* unitaries are embedded in the main function **dicke_state**. It receives as inputs the [QuantumVariable](../../reference/Core/QuantumVariable.rst) ``qv`` that we want to work on and an integer ``k``, which represents the desired Hamming-weight.
Here, we again invert the [jrange](../../reference/Jasp/Control%20Flow/jrange.rst) operator to represent the logic of the original paper.

In [20]:
from qrisp.jasp import check_for_tracing_mode
def dicke_state(qv,k):

    # jasp compatibility
    if check_for_tracing_mode():
        n = qv.size
    else:
        n = len(qv)

    # SCS cascade
    with invert():
        for index2 in jrange(k+1, n+1):
            split_cycle_shift(qv, index2, k,)
        #barrier(qv)
    with invert():
        for index in jrange(2,k+1):
            split_cycle_shift(qv, index, index-1, )
        #barrier(qv)

### How to use the [dicke_state()](../../reference/Primitives/DickeStates.rst) function

To run this code and properly generate the desired Dicke state, we have to make sure that the input state already has the desired Hamming-weight ``k`` in its trailing ``k`` qubits.

In other words, to receive $\ket{D_{2}^{4}}$ from calling ``dicke_state(qv,2)``, the ``qv`` has to in the $\ket{0011}$ state! 

We can therefore execute the following code:

In [21]:
from qrisp import QuantumVariable, x
# create the qv and put it in |0011> state
qv = QuantumVariable(4)
x(qv[2])
x(qv[3])
# call the dicke_state function
dicke_state(qv, 2)
# receive Dicke state with wt == 2
print(qv)


Simulating 5 qubits.. |                                                      | [  0%]

{'1100': 0.16666666666666666, '1010': 0.16666666666666666, '0110': 0.16666666666666666, '1001': 0.16666666666666666, '0101': 0.16666666666666666, '0011': 0.16666666666666666}


While this may be seen as an inhibition to the flexbility of the algorithm, this actually leads to some very useful behaviour;
The unitary which prepares $\ket{D_{2}^{4}}$ from $\ket{0011}$, lets name it $U_{2}^{4}$, also creates $\ket{D_{1}^{4}}$ from $\ket{0001}$!

More generally, a unitary $U_{k}^{n}$, which creates a given Hamming-weight $k$ state with $n$ total qubits, will also create any lower Hamming-weight state from the correct input state.

Mathematically speaking this means, with $n$ being a given number of qubits, $k$ a given Hamming-weight, and any other $l \leq k$. 

$ U_{k}^{n} (\ket{0}^{n-k} \otimes \ket{1}^{k} ) = \ket{D_{k}^{n}} \, \, \, \text{  and  } \, \, \,  U_{k}^{n} (\ket{0}^{n-l} \otimes\ket{1}^{l} )= \ket{D_{l}^{n}} $$


This is particularly useful for creating superpositions of different Hamming-weight Dicke states (see for example [the DQI algorithm (2024)](https://arxiv.org/abs/2408.08292) by S. Jordan et al.).

Consider the following example, where $\alpha \in (0,1)$

$$ U_{2}^{4} ( \sqrt{\alpha} \, \ket{0011} + \sqrt{1- \alpha} \, \ket{0001}  ) = \sqrt{\alpha} \, \ket{D_{2}^{4}} + \sqrt{1-\alpha} \, \ket{D_{1}^{4}} $$

Accordingly, we can execute the function from above on a QuantumVariable in superposition to receive the Dicke state in superposition!

In [22]:
# create the qv and put it in |0011> + |0001> state
qv = QuantumVariable(4)
h(qv[2])
x(qv[3])
# call the dicke_state function
dicke_state(qv, 2)
# receive superposition of Dicke states with Hamming-weight 1 and 2!
print(qv)

{'1000': 0.125002500050001, '0100': 0.125002500050001, '0010': 0.125002500050001, '0001': 0.125002500050001, '1100': 0.08333166663333266, '1010': 0.08333166663333266, '0110': 0.08333166663333266, '1001': 0.08333166663333266, '0101': 0.08333166663333266, '0011': 0.08333166663333266}


### Divide-and-Conquer approach

For the final algorithm in this tutorial let us investigate the [Divide-and-Conquer approach from Bärtschi et al.](https://arxiv.org/abs/2112.12435).

The idea here is to divide the whole Dicke state preparation procedure as follows: 

First, we separate the set of qubits into two sets.
Then, a smart prepreparation is conducted, after which the [dicke_state()](../../reference/Primitives/DickeStates.rst)-function is executed on each qubit set individually.
Finally, we fuse the qubit sets back together.

The main difficulty lays in choosing the correct weighting of states for the preparation step. For an in-depth explanation please refer to the original paper.
We will also make use of the function ``comb()``, a [JAX compatible](https://docs.jax.dev/en/latest/index.html) version of the binomial coeffient.

In [23]:

import jax
from jax.scipy.special import gammaln

@jax.jit
def comb(N, k):
    integ = jnp.uint16(jnp.round(jnp.exp(gammaln(N + 1) - gammaln(k + 1) - gammaln(N - k + 1))))
    return integ

In the following, we will keep it short. The [dicke_divide_and_conquer()](../../reference/Primitives/DickeStates.rst) function precomputes the correct weights, i.e. the [ry()](../../reference/Primitives/generated/qrisp.ry.rst)-gate angles to fan-out 
the amplitude information, and then applies a [cx()](../../reference/Primitives/generated/qrisp.ry.rst)-cascade. 
Afterwards, we apply the ``dicke_state()`` functions on the separted qubit set.
For the explanation of the [ry()](../../reference/Primitives/generated/qrisp.ry.rst)-angle calculation, we refer to the original paper. 

In [24]:
def dicke_divide_and_conquer(qv, k):

    # separate the QuantumVariable
    n = qv.size
    n_1 = jnp.floor(n/2)
    n_2 = n - n_1

    # divide step
    def dicke_divide(qv):
        l_xi = []
        rotation_angles = jnp.zeros(k)
        l_xi = jnp.zeros(k+1)

        # compute rotation angles
        for i1 in range(k+1):
            x_i = comb(n_1,i1)*comb(n_2,k-i1)
            l_xi = l_xi.at[i1].set(x_i)

        for i2 in range(k):
            temp_sum = jnp.sum(l_xi[i2:])
            rot_val = 2*jnp.acos(jnp.sqrt(l_xi[i2]/temp_sum))
            rotation_angles = rotation_angles.at[i2].set(rot_val)
        
        n_1h = n_1.astype(int)
        # apply the rotations
        ry(rotation_angles[0], qv[n_1h-1])
        # fan-out
        for i in range(1,k):
            with control(qv[n_1h-i]):
                ry(rotation_angles[i], qv[n_1h-i-1])
        
        x(qv[n-k:n])
        for i in range(k):
            cx(qv[n_1h-k+i], qv[-(i+1)])

    # call the divide step and the two conquer (dicke_state) steps.
    dicke_divide(qv)

    n_1a = n_1.astype(int)
    n_2a = n_2.astype(int)
    dicke_state(qv[:n_1a], k)
    dicke_state(qv[n-n_2a:], k)

Let's look at one final example on how to use this function with and without *Jaspification*.
We initiate a QuantumVariable with 7 qubits from this create the Dicke state with Hamming weight 3 with the [@terminal_sampling](../../reference/Jasp/Simulation%20Tools/Terminal%20Sampling.rst) decorator.

In [25]:
@terminal_sampling
def main():
    n = 7
    qv_1 = QuantumVariable(n)
    dicke_divide_and_conquer(qv_1, 3)

    return qv_1

res_jasp = main()
print(res_jasp)
    

{11: 0.028571435968790197, 13: 0.028571435968790197, 14: 0.028571435968790197, 19: 0.028571435968790197, 21: 0.028571435968790197, 22: 0.028571435968790197, 35: 0.028571435968790197, 37: 0.028571435968790197, 38: 0.028571435968790197, 67: 0.028571435968790197, 69: 0.028571435968790197, 70: 0.028571435968790197, 7: 0.02857143224350017, 25: 0.028571426655565127, 26: 0.028571426655565127, 41: 0.028571426655565127, 42: 0.028571426655565127, 49: 0.028571426655565127, 50: 0.028571426655565127, 56: 0.028571426655565127, 73: 0.028571426655565127, 74: 0.028571426655565127, 81: 0.028571426655565127, 82: 0.028571426655565127, 97: 0.028571426655565127, 98: 0.028571426655565127, 28: 0.028571421067630085, 44: 0.028571421067630085, 52: 0.028571421067630085, 76: 0.028571421067630085, 84: 0.028571421067630085, 88: 0.028571421067630085, 100: 0.028571421067630085, 104: 0.028571421067630085, 112: 0.028571421067630085}



Similarly, we can do the same thing without the decorator and wrapper, i.e. run it non JASP-mode

In [26]:
n = 7
qv_2 = QuantumVariable(n)
dicke_divide_and_conquer(qv_2, 3)

res = qv_2.get_measurement()
print(res)

{'1110000': 0.02857142857142857, '1101000': 0.02857142857142857, '1011000': 0.02857142857142857, '0111000': 0.02857142857142857, '1100100': 0.02857142857142857, '1010100': 0.02857142857142857, '0110100': 0.02857142857142857, '1001100': 0.02857142857142857, '0101100': 0.02857142857142857, '0011100': 0.02857142857142857, '1100010': 0.02857142857142857, '1010010': 0.02857142857142857, '0110010': 0.02857142857142857, '1001010': 0.02857142857142857, '0101010': 0.02857142857142857, '0011010': 0.02857142857142857, '1000110': 0.02857142857142857, '0100110': 0.02857142857142857, '0010110': 0.02857142857142857, '0001110': 0.02857142857142857, '1100001': 0.02857142857142857, '1010001': 0.02857142857142857, '0110001': 0.02857142857142857, '1001001': 0.02857142857142857, '0101001': 0.02857142857142857, '0011001': 0.02857142857142857, '1000101': 0.02857142857142857, '0100101': 0.02857142857142857, '0010101': 0.02857142857142857, '0001101': 0.02857142857142857, '1000011': 0.02857142857142857, '010001


An that's it! You have reached the end of tutorial and are now ready to prepare Dicke States with all of the state-of-the-art methodology!

#### References

[1] [Jeffery Yu et al., *Efficient preparation of Dicke states*, 2024, arXiv: 2411.03428](https://arxiv.org/abs/2411.03428)